In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
print(tf.test.gpu_device_name())

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('Suicide_Detection.csv', engine = 'python')

In [ ]:
data = data.sample(n = 100000, replace = True)

In [ ]:
data.head()

In [ ]:
data['class'].value_counts()

In [ ]:
data[data['class'] =='non-suicide']

In [ ]:
data_nondep = data[data['class'] == 'non-suicide']
data_nondep.shape

In [ ]:
data_dep = data[data['class']=='suicide']
data_dep.shape

In [ ]:
data_nondep_downsampled =data_nondep.sample(data_dep.shape[0], replace= True)
data_nondep_downsampled.shape

In [ ]:
data_balanced = pd.concat([data_dep, data_nondep_downsampled])

In [ ]:
data_balanced['class'].value_counts()

In [ ]:
data_balanced['suicide'] = data_balanced['class'].apply(lambda x: 1 if x=='suicide'  else 0)

In [ ]:
data_balanced.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train , x_test, y_train, y_test = train_test_split(data_balanced['text'], data_balanced['suicide'], stratify=data_balanced['suicide'] )

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
#bertlayer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
#nnl
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

#model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(x_train, y_train, epochs=10)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_predicted = model.predict(x_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
tweets = [
          'I am very happy today',
          'I am very poor',
          'Today was a bad day',
          'she broke my heart'
]
model.predict(tweets)